# Fingerprint segmentation module

In [1]:
import numpy as np
import cv2 as cv
import tensorflow as tf
from utils import show
from ipywidgets import interact
import time
from IPython.display import clear_output
import json

import os, sys
sys.path.append(os.path.dirname(os.path.realpath("."))) # add parent path to import the pyfing package of this project

import pyfing as pf

2023-10-09 14:16:32.379102: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 14:16:32.379176: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 14:16:32.379223: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 14:16:32.392192: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
PATH_FVC = '../../datasets/'
PATH_GT = '../../datasets/segmentationbenchmark/groundtruth/'
PATH_RES = '../../results/'

# Error analysis

For each database, fingerprints are sorted according to the error rate.

In [27]:
@interact(year=[2000,2002,2004], db=[1,2,3,4])
def choose_db(year, db):
    subset = 'a'
    with open(f'{PATH_TESTS}fvc{year}_db{db}_{subset}_res.txt', 'r') as filehandle:
        results = json.load(filehandle)

    results = sorted(results, key=lambda r: r[2][0])
    seg.parameters = segmentation.GradMagSegmentationParameters.load(f'{PATH_PARAMS}fvc{year}_db{db}_b_best_params.txt')
    
    @interact(i=(0, len(results)-1))
    def show_result(i=0):
        i, j, (err, dc) = results[i]
        img = cv.imread(f'{PATH_FVC}fvc{year}/db{db}_{subset}/{i}_{j}.png', cv.IMREAD_GRAYSCALE)
        gt = cv.bitwise_not(cv.imread(f'{PATH_GT}fvc{year}_db{db}_im_{i}_{j}seg.png', cv.IMREAD_GRAYSCALE))
        mask = seg.run(img)
        v_mask = cv.merge((gt, mask, cv.bitwise_not(img)))
        show((img,f'{i}_{j}'), (mask, 'Result'), (gt, 'Ground truth'), (v_mask, f'Err = {err:.2f}% DC = {dc:.3f}'))

interactive(children=(Dropdown(description='year', options=(2000, 2002, 2004), value=2000), Dropdown(descripti…